In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import scipy.stats as st
%cd C:\Users\luzhbin.aleksey\Desktop\analytics

C:\Users\luzhbin.aleksey\Desktop\analytics


**Загрузка и обработка данных:**

In [2]:
zvk = pd.read_csv('zvk_hlf_opr.csv')

In [3]:
zvk = zvk[(zvk.Product != 'auto-lombard') & (zvk.ProductType != 'ПСК ДОВЕРИЕ')]
zvk['Duration'] = list(map(int, zvk.Duration))
zvk = zvk[zvk.Duration <= 31]
print(zvk.shape[0])
zvk[:1]

634351


,NumZvk,DateZvk,Region,ProductType,Product,Status,ZvkSum,Duration,Name,Gender,...,ON,Age,hlfStatus,hlfDaysOfOverdue,oprZmSum,oprDate,oprODpb,oprDatepb,oprODpn,oprDatepn
0,6-01-015/OFF/19.104,2019-05-21,Ростовская,Новый клиент от 29.01.2019,offline,Отказано,11000.0,17,"ЛИТОВЧЕНКО АЛЕКСЕЙ СЕРГЕЕВИЧ,1997-10-20",Мужской,...,False,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Период времени, который включает в себя загруженный датафрейм:**

In [4]:
print(min(zvk.DateZvk), max(zvk.DateZvk))

2018-12-01 2019-05-21


**Функция обработки данных по просрочке (overdue):** 

In [144]:
def overdue(df, first, final):
    smpl_all = df[(df.DateZvk >= first) & (df.DateZvk <= final) & (pd.isnull(df.oprZmSum) == False)]
    smpl_all = smpl_all[(smpl_all.oprODpb == 0) | (smpl_all.oprODpn == 0)]
    smpl_all = smpl_all[(smpl_all.oprDatepb <= final) | (smpl_all.oprDatepn <= final)]
    smpl_all = smpl_all[(smpl_all.oprDate != smpl_all.oprDatepb) & (smpl_all.oprDate != smpl_all.oprDatepn)]
    
    smpl_ovr = smpl_all[smpl_all.hlfDaysOfOverdue > 3]
    
    n = smpl_all.shape[0]
    a = smpl_ovr.shape[0]
    p = a / n
    
    print(n, a)
    
    return [p, n, smpl_all, smpl_ovr]

**Функция обработки данных по погашенным займам (repayment):**

In [145]:
def repayment(df, first, final):
    smpl_all = df[(df.DateZvk >= first) & (df.DateZvk <= final) & (pd.isnull(df.oprZmSum) == False)]
    
    smpl_rep = smpl_all[(smpl_all.oprODpb == 0) | (smpl_all.oprODpn == 0)]
    smpl_rep = smpl_rep[(smpl_rep.oprDatepb <= final) | (smpl_rep.oprDatepn <= final)]
    smpl_rep = smpl_rep[(smpl_rep.oprDate != smpl_rep.oprDatepb) & (smpl_rep.oprDate != smpl_rep.oprDatepn)]
    
    n = smpl_all.shape[0]
    a = smpl_rep.shape[0]
    p = a / n
    
    print(n, a)
    
    return [p, n, smpl_all, smpl_rep]

**Статистика критерия:**

In [146]:
def stat(p1, p2, n1, n2):
    P = (p1 * n1 + p2 * n2) / (n1 + n2)
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1 / n1 + 1 / n2))

In [147]:
first1 = '2018-12-01'
final1 = '2019-01-31'

first2 = '2019-03-01'
final2 = '2019-04-30'

In [148]:
ovrd1 = overdue(zvk[zvk.ON == True], first1, final1) 
print('Доля вышедших на просрочку: {:.2f}%'.format(ovrd1[0] * 100))

ovrd2 = overdue(zvk[zvk.ON == True], first2, final2)
print('Доля вышедших на просрочку: {:.2f}%'.format(ovrd2[0] * 100))

12634 1067
Доля вышедших на просрочку: 8.45%
9017 624
Доля вышедших на просрочку: 6.92%


**Значение статистики критерия для overdue:**

In [149]:
zetstat = stat(ovrd1[0], ovrd2[0], ovrd1[1], ovrd2[1])
p = 2 * (1 - st.norm.cdf(np.abs(zetstat)))
print('p-value: {:.5f}'.format(p))

p-value: 0.00004


In [150]:
rep1 = repayment(zvk[zvk.ON == True], first1, final1) 
print('Доля погасивших займ: {:.2f}%'.format(rep1[0] * 100))

rep2 = repayment(zvk[zvk.ON == True], first2, final2)
print('Доля погасивших займ: {:.2f}%'.format(rep2[0] * 100))

24024 12634
Доля погасивших займ: 52.59%
20870 9017
Доля погасивших займ: 43.21%


**Значение статистики критерия для repayment:**

In [151]:
zetstat = stat(rep1[0], rep2[0], rep1[1], rep2[1])
p = 2 * (1 - st.norm.cdf(np.abs(zetstat)))
print('p-value: {:.5f}'.format(p))

p-value: 0.00000
